# NBA Games Preprocessor

This notebook takes games and transforms them into richer datasets.  This takes a reasonable amount of processing so this work is done upfront and commited to CSV.

## Configuration

In [3]:
use_parsed_data = True
output_parsed_data_filename = '../input/parsed_data.csv'

save_result_data = True
output_result_data_filename = '../output/result.csv'

calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
calc_ratings = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False

# Processor

In [4]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [5]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Load data from file system

In [4]:
#load games - json object
if use_parsed_data == True:
    games = pd.read_csv(output_parsed_data_filename)
    games = games.drop(['Unnamed: 0'], axis=1)
else:
    with open('../input/games.json') as gamesJsonRaw:
        d = json.load(gamesJsonRaw)

    games = json_normalize(d['games'])

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
games.sample(5)

,id,location,oddsBet365Away,oddsBet365Home,scoreAway,scoreHome,scoreQuarters,season,statsAway.Ast,statsAway.Blk,...,statsHome.seasonDefensiveRating,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating
3594,201004040IND,NaN,2.75,1.47,102,133,"[{'number': 1, 'scoreHome': 33, 'scoreAway': 2...",2009,24,4,...,103.95,105.75,2331.4,489.4,2477,529,106.25,108.09,106.80,107.89
931,32108,American Airlines Arena,13.50,1.04,98,103,"[{'number': 1, 'scoreHome': 30, 'scoreAway': 2...",2015,20,7,...,105.05,113.65,1383.6,456.8,1636,501,118.24,109.68,114.05,112.30
2099,34813,Oracle Arena,26.00,1.00,95,112,"[{'number': 1, 'scoreHome': 32, 'scoreAway': 2...",2016,25,3,...,117.08,117.08,277.4,277.4,315,315,113.55,113.55,108.15,108.15
8770,201503290WAS,NaN,1.68,2.25,99,91,"[{'number': 1, 'scoreHome': 20, 'scoreAway': 2...",2014,18,6,...,100.66,99.16,5862.8,491.0,6223,532,106.14,108.35,103.40,106.72
7747,201411070CHO,NaN,2.05,1.80,119,122,"[{'number': 1, 'scoreHome': 22, 'scoreAway': 2...",2014,28,3,...,100.83,105.26,1331.6,473.4,1386,501,104.09,105.83,102.06,98.01


In [6]:
#load stadiums - json object
with open('../input/stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [7]:
stadiums.head(5)

,lat,lng,team
0,33.757183,-84.396278,ATL
1,42.366281,-71.062266,BOS
2,40.682650,-73.974689,BRO
3,35.224519,-80.841053,CHA
4,41.880589,-87.674149,CHI


   ## Data Preperation

### Record the names of all stats - to be dropped later.

In [8]:
statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]

In [9]:
# Replace any NaN with 0 in the stats
games[statsColumns] = games[statsColumns].fillna(value=0)

### Set Valid Data Types

In [10]:
#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')

games['statsHome.FgAtt'] = games['statsHome.FgAtt'].astype('uint8')
games['statsHome.OffReb'] = games['statsHome.OffReb'].astype('uint8')
games['statsHome.Tov'] = games['statsHome.Tov'].astype('uint8')
games['statsHome.FtAtt'] = games['statsHome.FtAtt'].astype('uint8')
games['statsHome.Pts'] = games['statsHome.Pts'].astype('uint8')

games['statsAway.FgAtt'] = games['statsAway.FgAtt'].astype('uint8')
games['statsAway.OffReb'] = games['statsAway.OffReb'].astype('uint8')
games['statsAway.Tov'] = games['statsAway.Tov'].astype('uint8')
games['statsAway.FtAtt'] = games['statsAway.FtAtt'].astype('uint8')
games['statsAway.Pts'] = games['statsAway.Pts'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object

### Calculate the number of games played for season

In [11]:
if (use_parsed_data == False):

    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

    # Iterate through every team
    for team in games.teamAwayCode.unique():
        season = 0

        # Iterate through each game the team is present in.
        for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].sort_values(by='season', ascending=True).iterrows():
            if season != game.season:
                gameCount = 0
                homeGameCount = 0
                season = game.season

            gameCount += 1

            # Update game count for team - whether away or home.
            if game.teamAwayCode == team:
                data.loc[index]['gamesPlayedAway'] = gameCount
            else:
                data.loc[index]['gamesPlayedHome'] = gameCount

    data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
    data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')

    # Append the results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

In [12]:
games[['teamHomeCode', 'teamAwayCode', 'season', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,season,gamesPlayedHome,gamesPlayedAway
8128,LAL,PHX,2014,9,14
4739,SAC,GSW,2010,20,36
6364,MIN,ORL,2013,60,60
1301,BRO,CHA,2016,56,57
2417,WAS,MIA,2016,23,23


## Calculate the Win/Loss % for the Home team.

In [13]:
def calc_percentage(x, y):
    return 0 if y == 0 else (x / y)

In [14]:
if (use_parsed_data == False) & (calc_win_loss == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

### Calculate the results for the last 5 games for the Home team.

In [15]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the results for the last 5 games for the Away team.

In [16]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the average shooting % for the last 5 games

In [17]:
if (use_parsed_data == False) & (calc_ratings == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=[
        'statsHome.seasonTotalPossessions', 'statsHome.last5TotalPossessions',
        'statsHome.seasonTotalPoints', 'statsHome.last5TotalPoints',
        'statsHome.seasonOffensiveRating', 'statsHome.last5OffensiveRating',
        'statsHome.seasonDefensiveRating', 'statsHome.last5DefensiveRating',
        'statsAway.seasonTotalPossessions', 'statsAway.last5TotalPossessions',
        'statsAway.seasonTotalPoints', 'statsAway.last5TotalPoints',
        'statsAway.seasonOffensiveRating', 'statsAway.last5OffensiveRating',
        'statsAway.seasonDefensiveRating', 'statsAway.last5DefensiveRating'
    ])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get all previous games - Home
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False)

        totalPossessionsHome = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsHome = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredHome = allgames['statsHome.Pts'].sum()
        last5PointsScoredHome = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingHome = round(calc_percentage(totalPointsScoredHome, totalPossessionsHome) * 100, 2)
        last5OffensiveRatingHome = round(calc_percentage(last5PointsScoredHome, last5PossessionsHome) * 100, 2)

        totalPointsAgainstHome = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstHome = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingHome = round(calc_percentage(totalPointsAgainstHome, totalPossessionsHome) * 100, 2)
        last5DefensiveRatingHome = round(calc_percentage(last5PointsAgainstHome, last5PossessionsHome) * 100, 2)
        
        # Get all previous games - Away
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False)

        totalPossessionsAway = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsAway = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredAway = allgames['statsHome.Pts'].sum()
        last5PointsScoredAway = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingAway = round(calc_percentage(totalPointsScoredAway, totalPossessionsAway) * 100, 2)
        last5OffensiveRatingAway = round(calc_percentage(last5PointsScoredAway, last5PossessionsAway) * 100, 2)

        totalPointsAgainstAway = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstAway = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingAway = round(calc_percentage(totalPointsAgainstAway, totalPossessionsAway) * 100, 2)
        last5DefensiveRatingAway = round(calc_percentage(last5PointsAgainstAway, last5PossessionsAway) * 100, 2)

        data.loc[index] = [
            totalPossessionsHome, last5PossessionsHome, 
            totalPointsScoredHome, last5PointsScoredHome, 
            totalOffensiveRatingHome, last5OffensiveRatingHome,
            totalDefensiveRatingHome, last5DefensiveRatingHome,
            totalPossessionsAway, last5PossessionsAway, 
            totalPointsScoredAway, last5PointsScoredAway, 
            totalOffensiveRatingAway, last5OffensiveRatingAway,
            totalDefensiveRatingAway, last5DefensiveRatingAway
        ]

        # Update the row with the history
        # data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    data.sample(5)

    # Convert types
    # data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

    #games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)


In [18]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles


337.6783477035334

#### Save data to CSV to save on processing

In [19]:
if use_parsed_data == False:
    games.to_csv(output_parsed_data_filename, encoding='utf-8')

## Data Cleanse
Look at data types

In [20]:
games.dtypes

id                                   object
location                             object
oddsBet365Away                      float64
oddsBet365Home                      float64
scoreAway                             uint8
scoreHome                             uint8
scoreQuarters                        object
season                                int64
statsAway.Ast                         int64
statsAway.Blk                         int64
statsAway.BlkAgainst                  int64
statsAway.DefReb                      int64
statsAway.Ejections                   int64
statsAway.Fg2PtAtt                    int64
statsAway.Fg2PtMade                   int64
statsAway.Fg2PtPct                  float64
statsAway.Fg3PtAtt                    int64
statsAway.Fg3PtMade                   int64
statsAway.Fg3PtPct                  float64
statsAway.FgAtt                       uint8
statsAway.FgMade                      int64
statsAway.FgPct                     float64
statsAway.FoulFlag1             

Transform the Time column into a datetime object.

In [21]:
games["time"] = pd.to_datetime(games["time"])

### Convert the home and away teams to one-hot encoding 

In [22]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [23]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

Drop the id fields for teams.

In [24]:
#if (use_team_hot_encoding == True) | (use_team_home_away_hot_encoding == True) | (use_team_keys == False):
#    games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [25]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [26]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [27]:
games = games.drop(statsColumns, axis=1)

In [28]:
#gamesQuarters = json_normalize(data=d['games'], record_path='scoreQuarters', meta=['id'])
#gamesQuarters.head(10)

In [29]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [30]:
# Remove games scores ... not possible to predict the future, if you already know it :)
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [31]:
# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]

In [32]:
# Fill any NaN stats with 0
games.fillna(0, inplace=True)
#games['statsHome.Ejections'].fillna(0, inplace=True)
#games['statsAway.Ejections'].fillna(0, inplace=True)

In [33]:
# Show any NaN
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin


In [34]:
games.sample(3)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin
172,31348,9.50,1.07,2015,59,48,59,43,26,24,...,False,True,True,True,True,False,False,False,False,True
1314,34025,2.10,1.76,2016,56,55,54,29,28,16,...,True,True,False,False,False,True,True,False,False,True
1839,34553,1.62,2.40,2016,41,42,39,12,20,7,...,False,True,True,True,False,True,False,False,True,False


In [35]:
msk = np.random.rand(len(games)) < 0.8
msk

train = games[msk]
test = games[~msk]

Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)

#
X_test  = test.drop(["id", "homeWin"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((7288, 43), (7288,), (1780, 43))

In [36]:
# Reset the index
test.index = range(len(test.index))

In [37]:
X_test.sample(5)

,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,...,lastGame1WinAway,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway
9023,2.15,1.74,2015,88,8,88,47,43,27,45,...,False,False,True,False,True,False,False,True,True,True
2035,3.20,1.37,2016,8,10,8,1,3,1,5,...,False,False,False,True,False,False,True,False,False,False
6142,5.50,1.15,2012,44,13,39,20,19,12,20,...,True,True,True,True,False,False,True,False,True,True
3268,2.50,1.58,2009,7,7,5,2,2,2,3,...,False,True,True,True,True,False,True,True,True,True
7622,3.10,1.40,2013,55,52,56,36,26,20,30,...,False,False,False,True,True,False,False,True,True,False


### Logistic Regression

In [38]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

69.0

In [39]:
coeff_df = pd.DataFrame(games.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,oddsBet365Away,0.262221
21,percentageHomeWinAway,0.101411
36,lastGame2AtHomeAway,0.089989
38,lastGame3AtHomeAway,0.089049
11,percentageTotalWinHome,0.087459
28,lastGame3AtHomeHome,0.082019
31,lastGame5WinHome,0.077086
13,percentageAwayWinHome,0.071456
39,lastGame4WinAway,0.063964
23,lastGame1WinHome,0.057501


In [40]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_log = round((correct / total) * 100, 2)

print(correct_log, total, correct, incorrect)

67.58 1780 1203 577


### Support Vector Machines

In [41]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

97.079999999999998

In [42]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_svc = round((correct / total) * 100, 2)

print(correct_svc, total, correct, incorrect)

62.47 1780 1112 668


### k-Nearest Neighbors

In [43]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

79.340000000000003

In [44]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_knn = round((correct / total) * 100, 2)

print(correct_knn, total, correct, incorrect)

59.72 1780 1063 717


### Gaussian Naive Bayes

In [45]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

65.969999999999999

In [46]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_gaussian = round((correct / total) * 100, 2)

print(correct_gaussian, total, correct, incorrect)

65.51 1780 1166 614


### Perceptron

In [47]:
perceptron = Perceptron(max_iter = 5)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

59.850000000000001

In [48]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_perceptron = round((correct / total) * 100, 2)

print(correct_perceptron, total, correct, incorrect)

61.24 1780 1090 690


### Linear SVC

In [49]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

55.049999999999997

In [50]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_linear_svc = round((correct / total) * 100, 2)

print(correct_linear_svc, total, correct, incorrect)

54.33 1780 967 813


### Stochastic Gradient Descent

In [51]:
sgd = SGDClassifier(max_iter = 5)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

42.119999999999997

In [52]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_sgd = round((correct / total) * 100, 2)

print(correct_sgd, total, correct, incorrect)

40.9 1780 728 1052


### Decision Tree

In [53]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [54]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_decision_tree = round((correct / total) * 100, 2)

print(correct_decision_tree, total, correct, incorrect)

59.66 1780 1062 718


### Random Forest

In [55]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [56]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_random_forest = round((correct / total) * 100, 2)

print(correct_random_forest, len(Y_pred), total, correct, incorrect)

66.35 1780 1780 1181 599


## Model Evaluation Results

In [57]:
models = pd.DataFrame({
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Correct': [correct_svc, correct_knn, correct_log, 
              correct_random_forest, correct_gaussian, correct_perceptron, 
              correct_sgd, correct_linear_svc, correct_decision_tree],
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})
models.sort_values(by='Correct', ascending=False)

,Correct,Model,Score
2,67.58,Logistic Regression,69.00
3,66.35,Random Forest,100.00
4,65.51,Naive Bayes,65.97
0,62.47,Support Vector Machines,97.08
5,61.24,Perceptron,59.85
1,59.72,KNN,79.34
8,59.66,Decision Tree,100.00
7,54.33,Linear SVC,55.05
6,40.90,Stochastic Gradient Decent,42.12


#### Sample of ten incorrect predictions

In [58]:
result[(result['homeWin'] != result['forecast'])].sample(10)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin,forecast
1191,201303180UTA,6.00,1.13,2012,48,91,43,25,20,16,...,True,False,True,True,False,False,True,True,False,True
1010,201105090BOS,1.86,1.95,2010,21,39,21,13,10,7,...,True,True,False,True,False,True,False,True,False,True
1580,201412120BOS,3.40,1.33,2014,54,34,51,25,26,12,...,True,True,True,True,True,True,False,False,False,True
1039,201211270SAC,1.95,1.86,2012,35,35,34,14,17,10,...,False,False,True,True,True,False,False,False,False,True
234,32399,1.41,3.00,2015,40,40,38,7,23,6,...,True,True,False,True,True,False,True,True,True,False
1,31181,1.76,2.10,2015,62,82,59,30,27,17,...,True,False,True,True,True,False,True,False,True,False
1757,201505040HOU,2.80,1.45,2014,34,31,37,28,18,14,...,False,True,True,False,True,True,True,False,False,True
1017,201211030LAC,4.50,1.22,2012,61,66,56,37,29,21,...,True,True,False,True,False,False,False,True,False,True
251,34013,3.30,1.35,2016,57,57,56,30,25,16,...,True,False,True,True,True,True,True,False,False,True
134,31791,2.35,1.64,2015,84,62,88,60,43,35,...,False,True,True,False,True,False,False,True,False,True


## Calculate Profit

#### Number of games with predictions

In [59]:
len(result)

1780

### Iterate through all predicted games
* Gamble $50 on all games and calculate result

In [60]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(result)), columns=['winnings'])

bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.forecast == True) & (game.homeWin == True):
        winnings = (game.oddsBet365Home * betAmount);
    if (game.forecast == False) & (game.homeWin == False):
        winnings = (game.oddsBet365Away * betAmount)
    if (game.forecast != game.homeWin):
        winnings = 0 - betAmount

    data.loc[index] = [winnings]
    bank += winnings

result = pd.merge(result, data, left_index=True, right_index=True)
    
bank

54164.5

In [61]:
result[['id', 'winnings', 'homeWin', 'forecast', 'season', 'oddsBet365Home', 'oddsBet365Away', 'gamesPlayedHome', 'gamesPlayedAway', 'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome']].sample(30)

,id,winnings,homeWin,forecast,season,oddsBet365Home,oddsBet365Away,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome
189,32122,55.5,True,True,2015,1.11,7.00,39,37,44,28,21,18
839,201101050NJN,-50,True,False,2010,2.70,1.50,68,83,68,18,33,14
340,34455,67.5,False,False,2016,3.30,1.35,71,69,69,18,37,11
1699,201503160IND,-50,False,True,2014,1.47,2.75,26,53,36,15,14,8
29,31318,50.5,True,True,2015,1.01,18.00,61,50,61,45,28,26
1591,201412170MIA,-50,False,True,2014,1.57,2.55,50,8,43,24,25,15
292,34244,79,False,False,2016,2.50,1.58,62,62,60,17,31,11
746,201011030NJN,107.5,False,False,2010,1.74,2.15,58,55,56,18,30,14
761,201011090POR,57,True,True,2010,1.14,5.75,55,53,52,28,27,18
244,33976,55.5,False,False,2016,6.75,1.11,59,59,57,17,27,10


In [62]:
result['winnings'].sum()

54164.5

In [68]:
if save_result_data == True:
    result.to_csv(output_result_data_filename, encoding='utf-8')

In [64]:
#result[(result['forecast'] == False) & (result['oddsBet365Home'] < result['oddsBet365Away'])]

#### Test: Bet on the Home team regardless

In [65]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == True):
        winnings = (game.oddsBet365Home * betAmount);
    else:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

50589.5

#### Test: Bet on the Away team regardless

In [66]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == False):
        winnings = (game.oddsBet365Away * betAmount);
    else:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

31248.0

#### Test: Bet on the team with the lowest odds

In [67]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == False) & (game.oddsBet365Away < game.oddsBet365Home):
        winnings = (game.oddsBet365Away * betAmount);
    if (game.homeWin == True) & (game.oddsBet365Home < game.oddsBet365Away):
        winnings = (game.oddsBet365Home * betAmount);
    if winnings == 0:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

55795.0